# Covid-19-Trend-Predictor

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tools.eval_measures import rmse
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.layers import LSTM
from keras  import callbacks
from keras import optimizers
import tensorflow as tf


Using TensorFlow backend.


In [52]:
# data
df = pd.read_csv('../Timeseries-Data/timeseries_data.csv')
df.head()
# Preparing the data
columns_to_keep = ['Index','Location','Population_Density','Date','Min_Temp','Max_Temp','Lockdown','Num_Cases']
df = df[columns_to_keep]
df

,Index,Location,Population_Density,Date,Min_Temp,Max_Temp,Lockdown,Num_Cases
0,0.0,USA,94.0,1/22/2020,-4.0,6.0,0,1
1,1.0,USA,94.0,1/23/2020,-1.0,8.0,0,1
2,2.0,USA,94.0,1/24/2020,2.0,13.0,0,2
3,3.0,USA,94.0,1/25/2020,4.0,11.0,0,2
4,4.0,USA,94.0,1/26/2020,2.0,11.0,0,5
5,5.0,USA,94.0,1/27/2020,5.0,11.0,0,5
6,6.0,USA,94.0,1/28/2020,2.0,11.0,0,5
7,7.0,USA,94.0,1/29/2020,2.0,10.0,0,5
8,8.0,USA,94.0,1/30/2020,4.0,10.0,0,5
9,9.0,USA,94.0,1/31/2020,2.0,6.0,0,7


In [53]:
numeric_columns = ['Population_Density','Lockdown','Max_Temp','Min_Temp']
df_numeric = df[numeric_columns]
df_temp = df[['Min_Temp','Max_Temp']]
final_temp = []

df_numeric.head()
print('Min', np.min(df_numeric))
print('Max', np.max(df_numeric))
dataset = df_numeric.astype('float64')
# scaling
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)
temp_deno = (np.max(df['Max_Temp']) - np.min(df['Min_Temp']))
temp_num = np.min(df['Min_Temp'])
for ind in df['Index']:
    index = int(ind)
    df.at[index,'Population_Density'] = scaled_data[index][0]
    df.at[index,'Lockdown'] = scaled_data[index][1]
    df.at[index,'Min_Temp'] = (df.at[index,'Min_Temp'] - temp_num)/ temp_deno
    df.at[index,'Max_Temp'] = (df.at[index,'Max_Temp'] - temp_num)/ temp_deno
df

Min Population_Density    9.0
Lockdown              0.0
Max_Temp              5.0
Min_Temp             -8.0
dtype: float64
Max Population_Density    1202.0
Lockdown               100.0
Max_Temp                40.0
Min_Temp                26.0
dtype: float64


,Index,Location,Population_Density,Date,Min_Temp,Max_Temp,Lockdown,Num_Cases
0,0.0,USA,0.071249,1/22/2020,0.083333,0.291667,0,1
1,1.0,USA,0.071249,1/23/2020,0.145833,0.333333,0,1
2,2.0,USA,0.071249,1/24/2020,0.208333,0.437500,0,2
3,3.0,USA,0.071249,1/25/2020,0.250000,0.395833,0,2
4,4.0,USA,0.071249,1/26/2020,0.208333,0.395833,0,5
5,5.0,USA,0.071249,1/27/2020,0.270833,0.395833,0,5
6,6.0,USA,0.071249,1/28/2020,0.208333,0.395833,0,5
7,7.0,USA,0.071249,1/29/2020,0.208333,0.375000,0,5
8,8.0,USA,0.071249,1/30/2020,0.250000,0.375000,0,5
9,9.0,USA,0.071249,1/31/2020,0.208333,0.291667,0,7
